In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
from synthcity.utils.serialization import save_to_file, load_from_file

In [2]:
base_path = Path("/home/bcebere/Documents/datasets/cf")

train_static = pd.read_csv(base_path / "cf_static_train_data.csv.gz")
train_temporal_df = load_from_file(base_path / "temporal_train_data.pkl")
train_horizons = []
for tmp in train_temporal_df:
    train_horizons.append(tmp.index.values.tolist())

test_static = pd.read_csv(base_path / "cf_static_test_data.csv.gz")
test_temporal_df = load_from_file(base_path / "temporal_test_data.pkl")
test_horizons = []
for tmp in test_temporal_df:
    test_horizons.append(tmp.index.values.tolist())

drop_cols = ["death", "time_to_event", "id"]

train_T = train_static["time_to_event"].values
train_E = train_static["death"].values
train_static = train_static.drop(columns = drop_cols).fillna(-1)


test_T = test_static["time_to_event"].values
test_E = test_static["death"].values
test_static = test_static.drop(columns = drop_cols).fillna(-1)


train_static

,Gender,Smoking Status,Class I Mutation,Class II Mutation,Class III Mutation,Class IV Mutation,Class V Mutation,Class VI Mutation,DF508 Mutation,G551D Mutation
0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0
1,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0
2,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...
4632,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0
4633,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0
4634,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0
4635,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0


In [3]:
from sklearn.preprocessing import MinMaxScaler

features = []
for item in train_temporal_df:
    features.extend(item.columns)
features = np.unique(features)

def prep_temporal(df):
    temporal = []
    scale_cols = {
     "time" : [], 
     "BMI": [], 
     "Weight": [],
     "Best FEV1": [],
    }
    for item in df:
        for col in scale_cols:
            if col in item.columns:
                scale_cols[col].extend(item[col])
        scale_cols["time"].extend(item.index)
        
    scalers = {}
    for col in scale_cols:
        scalers[col] = MinMaxScaler().fit(np.asarray(scale_cols[col]).reshape(-1, 1))
    
    for item in df:
        for col in features:
            if col not in item:
                item[col] = np.nan
        item["time"] = item.index
        item = item.fillna(0)
        
        for col in scalers:
            item[col] = scalers[col].transform(item[col].values.reshape(-1, 1))
        
            
        temporal.append(item[features].astype(float))
    
    return temporal

    
train_temporal = prep_temporal(train_temporal_df)
test_temporal = prep_temporal(test_temporal_df)

In [4]:
import numpy as np
import pandas as pd

def process_temporal(in_static_data, in_temporal_data):
    all_temporal_data = []
    horizons = []
    
    for uid in in_static_data["id"].unique():
        local_temporal_idx = in_temporal_data["id"] == uid
        local_temporal = in_temporal_data[local_temporal_idx]
        columns = sorted(local_temporal["variable"].unique())
        times = sorted(local_temporal["time"].unique())

        temporal_data = pd.DataFrame([], columns = columns)


        for horizon in times:
            local_temporal_horizon_idx = local_temporal["time"] == horizon
            local_temporal_horizon = local_temporal[local_temporal_horizon_idx]

            horizon_data = pd.DataFrame(-1 * np.ones((1, len(columns))), columns = columns)

            proc = local_temporal_horizon[["variable", "value"]]
            proc.index = local_temporal_horizon["variable"]
            proc = proc.drop(columns = ["variable"])
            proc = proc.T.reset_index(drop = True)
            horizon_data[proc.columns] = proc

            temporal_data = pd.concat([temporal_data, horizon_data], ignore_index = True)
        temporal_data.index = times

        for col in columns:
            if col not in temporal_data:
                temporal_data[col] = -1
        horizons.append(temporal_data["time"])
        all_temporal_data.append(temporal_data[columns])
        
    assert len(all_temporal_data) == len(in_static_data)

    return all_temporal_data, horizons

#proc_test_temporal, proc_test_horizons = process_temporal(test_static, test_temporal)
#proc_train_temporal, proc_train_horizons = process_temporal(train_static, train_temporal)

In [5]:
from synthcity.plugins.core.dataloader import (
     TimeSeriesSurvivalDataLoader,
)
horizons = [0.25, 0.5, 0.75]
time_horizons = np.quantile(
             [t_ for t_, e_ in zip(train_T, train_E) if e_ == 1], horizons
).tolist()
    
survival_data_train = TimeSeriesSurvivalDataLoader(
    temporal_data=train_temporal,
    temporal_horizons=train_horizons,
    static_data=train_static,
    T=train_T,
    E=train_E,
    time_horizons=time_horizons,
)


survival_data_test = TimeSeriesSurvivalDataLoader(
    temporal_data=test_temporal,
    temporal_horizons=test_horizons,
    static_data=test_static,
    T=test_T,
    E=test_E,
    time_horizons=time_horizons,
)

[KeOps] Warning : Cuda libraries were not detected on the system ; using cpu only mode


In [6]:
train_static, train_temporal, train_horizons, train_T, train_E = survival_data_train.unpack(as_numpy = True, pad = True)

train_temporal[0]

array([[1.0, 0.0, 0.0, ..., 1.0, 1.0, 1.0],
       [1.0, 0.0, 0.0, ..., 1.0, 1.0, 1.0],
       [1.0, 0.0, 0.0, ..., 1.0, 1.0, 1.0],
       ...,
       [1.0, 0.0, 0.0, ..., 1.0, 1.0, 1.0],
       [1.0, 0.0, 0.0, ..., 1.0, 1.0, 1.0],
       [0.0, 0.0, 0.0, ..., 1.0, 1.0, 1.0]], dtype=object)

In [7]:
from synthcity.plugins.core.models.time_series_survival.benchmarks import (
     evaluate_ts_survival_model,
)


def eval_model(mod, **kwargs):
    n_folds = 3
    model = mod(**kwargs)
    
    train_static, train_temporal, train_horizons, train_T, train_E = survival_data_train.unpack(as_numpy = True)
    model.fit(train_static, train_temporal, train_horizons, train_T, train_E)

    
    test_static, test_temporal, test_horizons, test_T, test_E = survival_data_test.unpack(as_numpy = True)
    
    score = evaluate_ts_survival_model(
        [model] * n_folds, 
        test_static, test_temporal, test_horizons, test_T, test_E, 
        time_horizons, pretrained = True,
        n_folds = n_folds
    )
    return score


In [ ]:
from synthcity.plugins.core.models.time_series_survival import (
     DynamicDeephitTimeSeriesSurvival, rnn_modes, output_modes
 )

headers = ["Model", "C-Index", "Brier score"]
results = pd.DataFrame([], columns = headers)

for output_mode in output_modes:
    for base in rnn_modes:
        try:
            score = eval_model(DynamicDeephitTimeSeriesSurvival, rnn_type = base, output_type = output_mode)["str"]
        except BaseException:
            print("failed", base, output_mode)
            continue
        local_results = pd.DataFrame([[f"DynDeephit[{base} -> {output_mode}]", score["c_index"], score["brier_score"]]], columns = headers)
        results = pd.concat([results, local_results], ignore_index = True)
    
tabulate.tabulate(results, tablefmt='html')

failed GRU MiniRocket
failed LSTM MiniRocket
failed RNN MiniRocket
failed Transformer MiniRocket
failed Wavelet MiniRocket
failed GRU mWDNPlus
failed LSTM mWDNPlus
failed RNN mWDNPlus
failed Transformer mWDNPlus
failed Wavelet mWDNPlus


In [ ]:
from synthcity.plugins.core.models.time_series_survival import (
     CoxTimeSeriesSurvival,
 )

headers = ["Model", "C-Index", "Brier score"]
results = pd.DataFrame([], columns = headers)

for output_mode in output_modes:
    for base in rnn_modes:
        try:
            score = eval_model(CoxTimeSeriesSurvival, emb_rnn_type = base, emb_output_type = output_mode)["str"]
        except BaseException:
            print("failed", base, output_mode)
            continue
        local_results = pd.DataFrame([[f"CoxPH[{base} -> {output_mode}]", score["c_index"], score["brier_score"]]], columns = headers)
        results = pd.concat([results, local_results], ignore_index = True)

tabulate.tabulate(results, tablefmt='html')

In [ ]:
from synthcity.plugins.core.models.time_series_survival import (
     XGBTimeSeriesSurvival,
 )

headers = ["Model", "C-Index", "Brier score"]
results = pd.DataFrame([], columns = headers)

for output_mode in output_modes:
    for base in rnn_modes:
        try:
            score = eval_model(XGBTimeSeriesSurvival, emb_rnn_type = base, emb_output_type = output_mode)["str"]
        except BaseException:
            print("failed", base, output_mode)
            continue
        local_results = pd.DataFrame([[f"XGB[{base} -> {output_mode}]", score["c_index"], score["brier_score"]]], columns = headers)
        results = pd.concat([results, local_results], ignore_index = True)

tabulate.tabulate(results, tablefmt='html')